In [1]:
import numpy as np
import pandas as pd
import json
from utils import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from sklearn import preprocessing

In [2]:
df = pd.DataFrame(columns=['postText','truthClass'])
with open('data/clickbait_data.txt') as data:
    for sentence in data:
        positive_case = {'truthClass': 1, 'postText': sentence}
        df = df.append(positive_case, ignore_index = True)
df.head()

,postText,truthClass
0,Should I Get Bings\n,1
1,Which TV Female Friend Group Do You Belong In\n,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1


In [3]:
with open('data/non_clickbait_data.txt', encoding='utf-8') as data:
    for sentence in data:
        negative_case = {'truthClass': 0, 'postText': sentence}
        df = df.append(negative_case, ignore_index = True)
df = df.sample(frac=1)
df.head()

,postText,truthClass
11086,These Stills From Old Disney Cartoons Are Insa...,1
5883,15 Things Nobody Thinks To Tell You About Leav...,1
31053,Mark Martin Discovers There Is Life on the Roa...,0
3105,21 Things Everyone Who Does BodyCombat Will Un...,1
29424,"CEO of GM outlines plan for ""New GM"" after aut...",0


In [4]:
print(df[df['truthClass'] == 0].shape)
print(df[df['truthClass'] == 1].shape)

(16001, 2)
(15999, 2)


In [5]:
df['postText'].dropna(inplace=True)
df['postText'] = df['postText'].apply(cleanText)
df.head()

,postText,truthClass
11086,these stills from old disney cartoons are insa...,1
5883,15 things nobody thinks to tell you about leav...,1
31053,mark martin discovers there is life on the roa...,0
3105,21 things everyone who does bodycombat will un...,1
29424,ceo of gm outlines plan for new gm after auto ...,0


In [6]:
# length of titles with longest words
maxLen = maxLengthInPostText(df)
maxLen

26

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B/glove.6B.50d.txt')

In [8]:
train, test = train_test_split(df, test_size=0.1)
X_train, Y_train = np.array(train["postText"].tolist()), np.array(train["truthClass"].tolist())
X_test, Y_test = np.array(test["postText"].tolist()), np.array(test["truthClass"].tolist())
print(X_train.shape)
print(X_test.shape)

(28800,)
(3200,)


In [9]:
Indices = sentences_to_indices(X_train,word_to_index, maxLen)
print("X_Train_indices =\n", Indices.shape)

X_Train_indices =
 (28800, 26)


In [10]:
def ClickBait_BiLSTM(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)   
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    X = Bidirectional(LSTM(128, return_sequences=True))(embeddings)
    # dropout
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences=True))(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(128, return_sequences=False))(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 2 units
    X = Dense(2)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)  
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices, X) 
    return model

In [11]:
model = ClickBait_BiLSTM((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 26, 50)            20000050  
_________________________________________________________________
bidirectional (Bidirectional (None, 26, 256)           183296    
_________________________________________________________________
dropout (Dropout)            (None, 26, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 26, 256)           394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 26, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)              

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 2)
X_train_indices.shape

(28800, 26)

In [14]:
model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 64, shuffle=True)

Epoch 1/10
450/450 [==============================] - 55s 122ms/step - loss: 0.1448 - accuracy: 0.9432
Epoch 2/10
450/450 [==============================] - 56s 124ms/step - loss: 0.0935 - accuracy: 0.9658
Epoch 3/10
450/450 [==============================] - 55s 121ms/step - loss: 0.0772 - accuracy: 0.9720
Epoch 4/10
450/450 [==============================] - 53s 117ms/step - loss: 0.0680 - accuracy: 0.9747
Epoch 5/10
450/450 [==============================] - 48s 107ms/step - loss: 0.0597 - accuracy: 0.9788
Epoch 6/10
450/450 [==============================] - 33s 73ms/step - loss: 0.0525 - accuracy: 0.9806
Epoch 7/10
450/450 [==============================] - 32s 72ms/step - loss: 0.0452 - accuracy: 0.9832
Epoch 8/10
450/450 [==============================] - 33s 73ms/step - loss: 0.0371 - accuracy: 0.9869
Epoch 9/10
450/450 [==============================] - 33s 73ms/step - loss: 0.0338 - accuracy: 0.9870
Epoch 10/10
450/450 [==============================] - 33s 73ms/step - loss: 

In [15]:
def onehot_to_binary(data):
    binary = []
    for i in range(len(data)):
        binary.append(1) if data[i][1]> data[i][0] else binary.append(0)
    return binary

In [16]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,roc_auc_score, mean_squared_error 
y_train_pred_oh = model.predict(X_train_indices)
y_train_pred_binary = onehot_to_binary(y_train_pred_oh)
Y_train_binary = onehot_to_binary(Y_train_oh)

print("Training Error")
print('Accuracy %s' % accuracy_score(Y_train_binary, y_train_pred_binary))
print('Precision %s' % precision_score(Y_train_binary, y_train_pred_binary))
print('Recall %s' % recall_score(Y_train_binary, y_train_pred_binary))
print('F1 score: %s' % f1_score(Y_train_binary, y_train_pred_binary))

Training Error
Accuracy 0.9938541666666667
Precision 0.9897481766891427
Recall 0.9980573093734822
F1 score: 0.9938853767229764


In [17]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
y_pred_onehot = model.predict(X_test_indices)
y_pred_binary = onehot_to_binary(y_pred_onehot)

print("Testing Error")
print('Accuracy %s' % accuracy_score(Y_test, y_pred_binary))
print('Precision %s' % precision_score(Y_test, y_pred_binary))
print('Recall %s' % recall_score(Y_test, y_pred_binary))
print('F1 score: %s' % f1_score(Y_test, y_pred_binary))

Testing Error
Accuracy 0.973125
Precision 0.9612546125461254
Recall 0.9854981084489282
F1 score: 0.973225404732254


In [18]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.99      0.96      0.97      1614
           1       0.96      0.99      0.97      1586

    accuracy                           0.97      3200
   macro avg       0.97      0.97      0.97      3200
weighted avg       0.97      0.97      0.97      3200



In [19]:
# Utilize our model and test the real-world headlines
def test(headline):
    headline = cleanText(headline)
    headline_np = np.array([headline])
    indices = sentences_to_indices(headline_np, word_to_index, max_len = maxLen)
    #y_pred_onehot = lstm_model.predict(test_indices)
    y_pred_onehot = model.predict(indices)
    y_pred_binary = onehot_to_binary(y_pred_onehot)
    return True if y_pred_binary == [1] else False

headline = "Here are 10 things you may not know"
test(headline)

True